In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.constraints import max_norm as maxnorm # güncel kullanım şekilleri
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.constraints import max_norm # güncel kullanım şekilleri
from tensorflow.keras.optimizers.legacy import SGD  # güncel kullanım şekilleri
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense
# keras modlü eski sürümleri destekler bu yüzden keras yerine tensorflow kullanarak güncel kütüphane erişimi sağlanır
from tensorflow.keras.utils import to_categorical # np_utils kullanımı artık yok onun yerine numpy ile kullanılıp to_categorical ile aynı kullanımı veriyor
import numpy as np
from sklearn.metrics import confusion_matrix , classification_report
import os 


In [ ]:
# Veri seti klasörünün yolu (Data klasörünüzün yolu)
data_dir = "Datas"

# Veri setini yükleme ve ön işleme
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.15)

# Eğitim ve doğrulama verilerini yükleme
input_width, input_height = 150, 150
batch_size = 32
num_classes = 4  # Toplam sınıf sayısı

train_data = datagen.flow_from_directory(data_dir, target_size=(input_width, input_height),
                                        class_mode='categorical', batch_size=batch_size, subset='training')
validation_data = datagen.flow_from_directory(data_dir, target_size=(input_width, input_height),
                                             class_mode='categorical', batch_size=batch_size, subset='validation')

# Eğitim sonrası test için kullanabilirsiniz
test_data = datagen.flow_from_directory(data_dir, target_size=(input_width, input_height), class_mode='categorical',
                                       batch_size=batch_size)
# CNN modelini oluşturun
model = tf.keras.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(input_width, input_height, 3), padding='same',
                 kernel_constraint=max_norm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=max_norm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=max_norm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# SGD optimizer'ı kullanarak modeli derleyin
sgd = SGD(learning_rate=0.01,momentum=0.9,decay=(0.01/25),nesterov=False)
model.compile(loss='categorical_crossentropy', 
              optimizer=sgd,
              metrics=['accuracy'])

# Modeli eğitin
model.fit(train_data, validation_data=validation_data, epochs=50)


Found 852 images belonging to 4 classes.
Found 148 images belonging to 4 classes.
Found 1000 images belonging to 4 classes.
Epoch 1/50
27/27 [==============================] - 41s 1s/step - loss: 1.4955 - accuracy: 0.3275 - val_loss: 1.3603 - val_accuracy: 0.3581
Epoch 2/50
27/27 [==============================] - 34s 1s/step - loss: 1.2726 - accuracy: 0.4143 - val_loss: 1.3102 - val_accuracy: 0.3986
Epoch 3/50
27/27 [==============================] - 34s 1s/step - loss: 1.2126 - accuracy: 0.4718 - val_loss: 1.2637 - val_accuracy: 0.4122
Epoch 4/50
27/27 [==============================] - 35s 1s/step - loss: 1.1659 - accuracy: 0.4636 - val_loss: 1.1204 - val_accuracy: 0.5068
Epoch 5/50
27/27 [==============================] - 34s 1s/step - loss: 1.1562 - accuracy: 0.4906 - val_loss: 1.1168 - val_accuracy: 0.4392
Epoch 6/50
27/27 [==============================] - 34s 1s/step - loss: 1.1231 - accuracy: 0.4965 - val_loss: 1.1742 - val_accuracy: 0.4257
Epoch 7/50
27/27 [==================

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 dropout (Dropout)           (None, 150, 150, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 150, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 75, 75, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 180000)            0         
                                                                 
 dense (Dense)               (None, 512)               92160512  
                                                        

In [7]:
_,acc=model.evaluate(test_data)
print(acc*100)    

5/5 [==============================] - 1s 113ms/step - loss: 1.9406 - accuracy: 0.4662
46.621620655059814


In [8]:
import warnings
warnings.filterwarnings("ignore")
model.save('cnn.h5')